# NEWSPAPER TITLE CLASSIFICATION BASED ON KNN, KMEANS AND DECISION TREE

To have general view & data structure of Project, refer the `Readme.md` of this Project and general structure of project

![General Structure](general_structure.png)

## 1. IMPORT LIBRARY

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import re
import nltk # use in case

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

## 2. DATA FEATURE EXTRACTION